<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo:XGBoost#

Ensamble de tipo boosting.

##Inicialización##


In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
)

In [5]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 95 (delta 48), reused 41 (delta 16), pack-reused 0
Unpacking objects: 100% (95/95), done.
/content/fiumark/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


##Pre procesamiento##

In [6]:
pd.options.display.max_columns = None
df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [11]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': np.arange(50,300,50),
    'max_depth': np.arange(1, 10),
    'min_child_weight': np.arange(1, 3)
}

clf = XGBClassifier()

rgscv = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [8]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8645031179892773
Best params {'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 2}


##Evaluación del modelo##

In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.79      0.96      0.86        70
         1.0       0.92      0.65      0.76        51

    accuracy                           0.83       121
   macro avg       0.85      0.80      0.81       121
weighted avg       0.84      0.83      0.82       121



##Predicción##

In [10]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
prediction = pd.DataFrame(rgscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('xgboost.csv')
pred_formatted

,id_usuario,volveria
0,650,0
1,234,0
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,1
